In [ ]:
#@title 🔑 OpenRouter API Key Provisioning Tool { display-mode: "form" }
#@markdown ### Welcome! This tool helps you create OpenRouter API keys for multiple users.
#@markdown
#@markdown **📋 What you need:**
#@markdown - A provisioning API key from [OpenRouter](https://openrouter.ai/settings/provisioning-keys) (store it in Colab Secrets for security)
#@markdown - Either a CSV with user data OR auto-generate placeholder users
#@markdown - CSV must have a **Name** column; **Email**, **Surname**, and **Budget** are optional
#@markdown
#@markdown **🚀 How to use:**
#@markdown 1. Run this cell
#@markdown 2. Choose data input mode (Upload CSV or Generate Users)
#@markdown 3. Set default budget
#@markdown 4. Create keys and download results
#@markdown
#@markdown **🔐 Security Tip:** Use Colab Secrets for your provisioning key (`OPENROUTER_PROVISIONING_KEY`).
#@markdown
#@markdown ---

# Install and import required libraries
import sys

# Install dependencies
try:
    import pandas as pd
except ImportError:
    !pip install -q pandas
    import pandas as pd

try:
    import ipywidgets as widgets
except ImportError:
    !pip install -q ipywidgets
    import ipywidgets as widgets

try:
    from tqdm import tqdm
except ImportError:
    !pip install -q tqdm
    from tqdm import tqdm

# Standard library imports
import os
import io
import time
import requests
from datetime import datetime
from pathlib import Path
from IPython.display import display, HTML, clear_output

# Check if running in Google Colab
try:
    from google.colab import files, userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    userdata = None

print("✅ All libraries loaded successfully\n")

# ============================================================================
# CORE FUNCTIONS
# ============================================================================

def _to_float_positive(value, default=None):
    try:
        v = float(value)
        return v if v > 0 else default
    except (TypeError, ValueError):
        return default


def provision_single_key(provision_key, name, label, limit):
    """Create a single OpenRouter API key using the provisioning API."""
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    try:
        response = requests.post(
            url=BASE_URL,
            headers={
                "Authorization": f"Bearer {provision_key}",
                "Content-Type": "application/json"
            },
            json={
                "name": name,
                "label": label,
                "limit": limit
            },
            timeout=30
        )
        if response.status_code == 201:
            return response.json(), None
        else:
            return None, f"HTTP {response.status_code}: {response.text}"
    except Exception as e:
        return None, str(e)


def provision_keys_bulk(df, provision_key, default_budget):
    """Provision OpenRouter API keys for all users in the DataFrame.
    Expects lowercase columns: name, email?, surname?, budget?."""
    results = {'successful': 0, 'failed': 0, 'errors': []}
    output_df = df.copy()
    output_df.columns = [str(c).strip().lower() for c in output_df.columns]

    if 'name' not in output_df.columns:
        raise ValueError("CSV must contain a 'name' column (case-insensitive)")

    if 'budget' not in output_df.columns:
        output_df['budget'] = default_budget
    if 'api_key' not in output_df.columns:
        output_df['api_key'] = None
    if 'created' not in output_df.columns:
        output_df['created'] = None

    timestamp_prefix = datetime.utcnow().strftime("%Y%m%d_%H%M%S")

    for index, row in tqdm(output_df.iterrows(), total=len(output_df), desc="Creating keys"):
        name_parts = [str(row.get('name', '')).strip()]
        if 'surname' in output_df.columns and pd.notna(row.get('surname')):
            name_parts.append(str(row.get('surname')).strip())
        full_name = "_".join([p for p in name_parts if p]) or f"user_{index+1}"
        email = str(row.get('email')).strip() if 'email' in output_df.columns and pd.notna(row.get('email')) else full_name

        budget = _to_float_positive(row.get('budget'), default_budget)
        if budget is None:
            budget = default_budget
        output_df.at[index, 'budget'] = budget

        key_name = f"APIKey_{timestamp_prefix}_{full_name}"
        key_label = f"{timestamp_prefix}_{email}"

        key_data, error = provision_single_key(provision_key, key_name, key_label, budget)
        created_at = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'

        if key_data and 'key' in key_data:
            output_df.at[index, 'api_key'] = key_data['key']
            output_df.at[index, 'created'] = created_at
            results['successful'] += 1
        else:
            output_df.at[index, 'created'] = created_at  # record attempt time even if failed
            results['failed'] += 1
            results['errors'].append({'name': full_name, 'error': error or 'Unknown error'})
        time.sleep(0.2)

    return output_df, results


def save_and_download_csv(df, filename):
    out_df = df.copy()
    out_df.columns = [str(c).strip().lower() for c in out_df.columns]
    filepath = f"/content/{filename}" if IN_COLAB else filename
    out_df.to_csv(filepath, index=False)
    if IN_COLAB:
        files.download(filepath)
    return filepath

# ============================================================================
# GUI CREATION
# ============================================================================
provisioning_key_from_secrets = None
if IN_COLAB and userdata is not None:
    try:
        provisioning_key_from_secrets = userdata.get('OPENROUTER_PROVISIONING_KEY')
    except Exception:
        provisioning_key_from_secrets = None

if provisioning_key_from_secrets:
    provision_key_widget = widgets.Text(
        value='',
        placeholder='✅ PROVISIONING KEY LOADED - GOOD TO GO',
        description='Provision Key:',
        disabled=True,
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )
else:
    provision_key_widget = widgets.Password(
        value='',
        placeholder='⚠️ ENTER YOUR OWN PROVISIONING KEY',
        description='Provision Key:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px')
    )

# Mode selection: Upload vs Generate
mode_selector = widgets.ToggleButtons(
    options=[('Upload CSV', 'upload'), ('Generate Users', 'generate')],
    description='Mode:',
    button_style='',
    style={'description_width': '80px'}
)

# Upload widget
upload_button = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    button_style='info',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='600px')
)

# Generation widgets
generate_prefix = widgets.Text(
    value='participant',
    description='Prefix:',
    placeholder='prefix',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='250px')
)

generate_count = widgets.BoundedIntText(
    value=5,
    min=1,
    max=1000,
    step=1,
    description='Count:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='200px')
)

generate_button = widgets.Button(
    description='Generate Users',
    button_style='primary',
    layout=widgets.Layout(width='200px')
)

csv_info = widgets.HTML(
    value='<p style="margin-left: 130px; color: #666; font-size: 0.85em;">CSV needs <b>Name</b>. Optional: <b>Email</b>, <b>Surname</b>, <b>Budget</b>. Case-insensitive.</p>'
)

gen_info = widgets.HTML(
    value='<p style="margin-left: 10px; color: #666; font-size: 0.85em;">Will create rows name=prefix_0..prefix_N-1</p>'
)

budget_widget = widgets.BoundedFloatText(
    value=1.00,
    min=0.01,
    max=10000.00,
    step=1.00,
    description='Def. Budget:',
    style={'description_width': '90px'},
    layout=widgets.Layout(width='220px')
)

budget_info = widgets.HTML(
    value='<span style="margin-left: 5px; color: #666; font-size: 0.85em;">USD per key (fallback)</span>'
)

create_button = widgets.Button(
    description='🔑 CREATE KEYS',
    button_style='success',
    layout=widgets.Layout(width='200px', height='42px'),
    style={'font_weight': 'bold'}
)

output_area = widgets.Output()

uploaded_df = None  # Shared state

# ============================================================================
# EVENT HANDLERS
# ============================================================================

def on_upload_change(change):
    global uploaded_df
    with output_area:
        clear_output()
        if mode_selector.value != 'upload':
            print("ℹ️ Switch to 'Upload CSV' mode to use file upload.")
            return
        if len(upload_button.value) == 0:
            print("⚠️ No file uploaded yet.")
            return
        try:
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            uploaded_df = pd.read_csv(io.BytesIO(content))
            uploaded_df.columns = [str(c).strip().lower() for c in uploaded_df.columns]
            if 'name' not in uploaded_df.columns:
                print("❌ ERROR: CSV must contain a 'Name' column (case-insensitive)")
                print(f"Found columns: {list(uploaded_df.columns)}")
                uploaded_df = None
                return
            print(f"✅ CSV loaded: {len(uploaded_df)} rows")
            print(f"Columns (normalized): {list(uploaded_df.columns)}")
            display(uploaded_df.head())
            if 'budget' in uploaded_df.columns:
                valid_budgets = uploaded_df['budget'].apply(lambda x: _to_float_positive(x) is not None).sum()
                print(f"💰 Budget column: {valid_budgets}/{len(uploaded_df)} valid")
            else:
                print("💰 No budget column — default budget will be used for all keys.")
        except Exception as e:
            print(f"❌ Error loading CSV: {e}")
            uploaded_df = None


def on_generate_click(button):
    global uploaded_df
    with output_area:
        clear_output()
        if mode_selector.value != 'generate':
            print("ℹ️ Switch to 'Generate Users' mode to create placeholder users.")
            return
        prefix = generate_prefix.value.strip() or 'user'
        count = generate_count.value
        default_budget = budget_widget.value
        names = [f"{prefix}_{i}" for i in range(count)]
        budgets = [default_budget for _ in range(count)]
        uploaded_df = pd.DataFrame({'name': names, 'budget': budgets})
        print(f"✅ Generated {count} placeholder users with prefix '{prefix}'")
        print("\n📋 Preview (first 5 rows):")
        display(uploaded_df.head())
        print("\n💡 Default budget applied. You can now press CREATE KEYS.")


def on_mode_change(change):
    with output_area:
        clear_output()
        print(f"ℹ️ Mode switched to: {mode_selector.value}. Follow the instructions above.")


def on_create_click(button):
    global uploaded_df
    with output_area:
        clear_output()
        provision_key = provisioning_key_from_secrets or provision_key_widget.value
        if not provision_key or provision_key.strip() == '':
            print("❌ ERROR: Provisioning key is required.")
            return
        if uploaded_df is None:
            print("❌ ERROR: No data available. Upload a CSV or generate users first.")
            return
        uploaded_df.columns = [str(c).strip().lower() for c in uploaded_df.columns]
        default_budget = budget_widget.value
        print("🚀 PROVISIONING CONFIGURATION")
        print("="*60)
        print(f"Users: {len(uploaded_df)}")
        print(f"Default budget: ${default_budget:.2f}")
        if 'budget' in uploaded_df.columns:
            total_est = 0.0
            for _, row in uploaded_df.iterrows():
                b = _to_float_positive(row.get('budget'), default_budget)
                total_est += b if b is not None else default_budget
            print(f"Estimated total budget: ${total_est:.2f}")
        else:
            print(f"Estimated total budget: ${default_budget * len(uploaded_df):.2f}")
        print("="*60)
        print("\n⏳ Starting key provisioning...\n")
        try:
            result_df, results = provision_keys_bulk(uploaded_df, provision_key, default_budget)
            print("\n" + "="*60)
            print("📊 PROVISIONING RESULTS")
            print("="*60)
            print(f"✅ Successful: {results['successful']} / {len(result_df)}")
            print(f"❌ Failed: {results['failed']} / {len(result_df)}")
            actual_total = result_df[result_df['api_key'].notna()]['budget'].sum()
            print(f"💰 Total budget allocated: ${actual_total:.2f}")
            print("="*60)
            preview = result_df.copy()
            preview['status'] = preview['api_key'].apply(lambda v: '✅ created' if pd.notna(v) and str(v).strip() != '' else '❌ failed')
            cols_to_show = [c for c in ['name', 'email', 'budget', 'created', 'status'] if c in preview.columns]
            if cols_to_show:
                print("\n📋 Preview (first 5 rows):")
                display(preview[cols_to_show].head())
            if results['errors']:
                print("\n⚠️ FAILED KEY CREATIONS:")
                for i, err in enumerate(results['errors'], 1):
                    print(f"{i}. {err['name']}: {err['error']}")
            if results['successful'] > 0:
                ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
                filename = f"provisioned_keys_{ts}.csv"
                print("\n💾 Saving results...")
                filepath = save_and_download_csv(result_df, filename)
                print(f"✅ Saved to: {filepath}")
                if IN_COLAB:
                    print("📥 Download started automatically.")
                print("\n⚠️ SECURITY REMINDER:")
                print("• Keep the CSV file with API keys secure")
                print("• Distribute keys via encrypted channels only")
                print("• Never commit API keys to public repositories")
            else:
                print("\n❌ No keys were created successfully.")
        except Exception as e:
            print(f"\n❌ CRITICAL ERROR: {e}")
            import traceback
            print("\nFull trace:")
            print(traceback.format_exc())

# Attach handlers
mode_selector.observe(on_mode_change, names='value')
upload_button.observe(on_upload_change, names='value')
create_button.on_click(on_create_click)
generate_button.on_click(on_generate_click)

# ============================================================================
# DISPLAY GUI
# ============================================================================
print("🎨 OpenRouter API Key Provisioning Interface\n")

mode_box = widgets.VBox([
    widgets.HTML('<h3>📁 Step 2: Data Input</h3>'),
    mode_selector,
    widgets.HTML('<div style="height:6px"></div>'),
    widgets.VBox([
        widgets.HTML('<b>Upload CSV</b>'),
        upload_button,
        csv_info
    ]),
    widgets.VBox([
        widgets.HTML('<b>Or Generate Users</b>'),
        widgets.HBox([generate_prefix, generate_count]),
        widgets.HBox([generate_button, gen_info])
    ])
])

budget_box = widgets.VBox([
    widgets.HTML('<h3>💰 Step 3: Set Default Budget</h3>'),
    widgets.HBox([budget_widget, budget_info])
])

actions_box = widgets.VBox([
    widgets.HTML('<h3>🚀 Step 4: Create Keys</h3>'),
    create_button
])

ui = widgets.VBox([
    widgets.HTML('<h3>🔑 Step 1: Provisioning Key</h3>'),
    provision_key_widget,
    mode_box,
    budget_box,
    actions_box,
    widgets.HTML('<hr>'),
    output_area
])

display(ui)